3 exercício

# Aplicando Grid:


In [ ]:
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# modulos referentes ao AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split

from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

df = pd.read_csv("german_credit_data.csv", index_col=0)

X = df.drop(columns="Risk")
y = df["Risk"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
categorical_features = df.select_dtypes(include = ["object"]).columns
numerical_features = df.select_dtypes(exclude = ["object"]).columns
categorical_features = categorical_features.drop("Risk")
df_num = df[numerical_features]
df_cat = df[categorical_features]
df_num = df_num.fillna(df_num.median()) #preenchendo os valores numéricos com a mediana, ao invés da média

df_cat = pd.get_dummies(df_cat) #dummies
X = pd.concat([df_cat,df_num],axis=1)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state= 0)

scaler=StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.fit_transform(X_test)

Substitui o pipeline pelos métodos que estou mais acostumado

In [ ]:
from sklearn.tree import DecisionTreeClassifier
param_grid = {
    'n_estimators': [50,100,150],
    'learning_rate': np.arange(0.1, 1, 0.1),
}
base_estimator = DecisionTreeClassifier()

# Define the AdaBoost classifier
adaboost_clf = AdaBoostClassifier(random_state=42)

grid_search = GridSearchCV(adaboost_clf, param_grid, cv=6, scoring='f1_weighted', verbose=1, n_jobs=-1)
grid_result = grid_search.fit(X_train, y_train)

# Get the best parameters found by Grid Search
best_params = grid_result.best_params_
print("Best Parameters:", best_params)

# Get the best model
best_model = grid_result.best_estimator_

# Evaluate the best model on the test set
print('Resultados para o Treino')
predictions_grid = best_model.predict(X_train)
print(classification_report(y_train, predictions_grid))



print('Resultados para o Teste')
predictions_grid = best_model.predict(X_test)
print(classification_report(y_test, predictions_grid))

Fitting 6 folds for each of 27 candidates, totalling 162 fits
Best Parameters: {'learning_rate': 0.7000000000000001, 'n_estimators': 50}
Resultados para o Treino
              precision    recall  f1-score   support

         bad       0.70      0.48      0.57       242
        good       0.80      0.91      0.85       558

    accuracy                           0.78       800
   macro avg       0.75      0.70      0.71       800
weighted avg       0.77      0.78      0.77       800

Resultados para o Teste
              precision    recall  f1-score   support

         bad       0.59      0.41      0.48        58
        good       0.79      0.88      0.83       142

    accuracy                           0.74       200
   macro avg       0.69      0.65      0.66       200
weighted avg       0.73      0.74      0.73       200



# Aplicando Regressão

Para regressão irei usar o mesmo data set da aula dois, e repetir o processo, e no final trocar para o AdaBoosting.

Isso pois a target do data set german_credit_data é descritiva, nao combina com a regressão.

In [ ]:
# dropando colunas somente colunas numéricas com modo de correlação >0,6
df = pd.read_csv("house_prices.csv")
colunas = df.corr().index[abs(df.corr()["SalePrice"])>0.6] #colunas numéricas com mais correlação
# checando nulos
categorical_features = df[colunas].columns
numerical_features = df.select_dtypes(exclude = ["object"]).columns
numerical_features = numerical_features.drop("SalePrice")
df_num = df[numerical_features]
df_cat = df[categorical_features]

total = df_num.isnull().sum().sort_values(ascending=False)
missing_data = pd.concat([total], axis=1, keys=['Total'])
df_num = df_num.fillna(df_num.median()) #preenchendo os valores numéricos com a mediana, ao invés da média
df.SalePrice = np.log1p(df.SalePrice ) # aplicando o log no target
y = df.SalePrice
df_cat = pd.get_dummies(df_cat) #dummies
X = pd.concat([df_cat,df_num],axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state= 0)
scaler=StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.fit_transform(X_test)



<ipython-input-39-40fb51892c0f>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  colunas = df.corr().index[abs(df.corr()["SalePrice"])>0.6] #colunas numéricas com mais correlação


In [ ]:

from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
model=AdaBoostRegressor(random_state=42)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_pred_test = model.predict(X_test)

  # predições de treino

print("\nMétricas de avaliação de treino")
print(f"R2: { r2_score(y_train, y_train_pred):.2f}")
print(f"MAE: { mean_absolute_error(y_train, y_train_pred):.2f}")
print(f"RMSE: { np.sqrt(mean_squared_error(y_train, y_train_pred)):.2f}")

    # predições de teste


print("\nMétricas de avaliação de teste")
print(f"R2: { r2_score(y_test, y_pred_test):.2f}")
print(f"MAE: { mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"RMSE: { np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")


Métricas de avaliação de treino
R2: 1.00
MAE: 0.02
RMSE: 0.03

Métricas de avaliação de teste
R2: 0.99
MAE: 0.03
RMSE: 0.04
